In [0]:
df = spark.read.format('delta').load('abfss://bronze@databricksetestg.dfs.core.windows.net/orders')
display(df)

In [0]:
df.printSchema()

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
df = df.withColumnRenamed("_rescued_data","rescued_data")
df = df.drop("rescued_data")
df = df.withColumn('order_date',to_timestamp(col('order_date')))
df = df.withColumn('year',year(col('order_date')))
display(df)


In [0]:
from pyspark.sql.window import Window
df_window = df.withColumn("dense_flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df_window = df_window.withColumn("rank_flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df_window = df_window.withColumn("row_number_flag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df_window)

### ***classes***

In [0]:
class windows:

    def dense_rank(self,df1):
        df_dense_rank = df1.withColumn("dense_rank",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_dense_rank
    def rank(self,df1):
        df_rank = df1.withColumn("rank",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_rank
    def row_number(self,df1):
        df_row_number = df1.withColumn("row_number",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_row_number

In [0]:
obj = windows()

In [0]:
df_new = df
display(df_new)

In [0]:
df_result = obj.dense_rank(df_new)


In [0]:
display(df_result)

In [0]:
df.write.format('delta').mode('overwrite').save('abfss://silver@databricksetestg.dfs.core.windows.net/orders')

In [0]:
df = spark.read.format('delta').load('abfss://silver@databricksetestg.dfs.core.windows.net/orders')
display(df)

In [0]:
%sql
CREATE SCHEMA prem_catalog.silver

In [0]:
%sql
CREATE TABLE IF NOT EXISTS prem_catalog.silver.orders_silver
USING DELTA
LOCATION 'abfss://silver@databricksetestg.dfs.core.windows.net/orders'